In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

weight = "PWGTP"
housing_identifier = "SERIALNO"
person_identifier = "SPORDER"

def import_census_data(vars, target, year, limiters): #acs data
  '''
  limiters: specified values to limit data requested
  '''
  curr = 0
  base_request = "https://api.census.gov/data/" + year + "/acs/acs1/pums?get=" + weight + "," + target + "," + housing_identifier + "," + person_identifier
  request = base_request

  #limit request
  limiter = ""
  for l in limiters:
    if l == "ST":
      limiter += "&ucgid=" + limiters[l]
    else:
      limiter += "&" + l + "=" + limiters[l]
      vars.remove(l)
  request += limiter

  #remove base vars
  vars.remove(target)
  vars.remove(weight)
  vars.remove(housing_identifier)
  vars.remove(person_identifier)

  #get target/weight
  response = requests.get(request)
  json_data = json.dumps(response.json())
  df = pd.read_json(json_data)
  df = df.rename(columns=df.iloc[0]).loc[1:].reset_index()
  df = df[[target, weight, housing_identifier, person_identifier]]

  target_col = df[target]
  weight_col = df[weight]
  housing_identifier_col = df[housing_identifier]
  person_identifier_col = df[person_identifier]

  #requesting all data
  while curr < len(vars):
    df, curr = request_vars_and_merge(vars, df, base_request, limiter, target, 4, curr,
                                target_col, weight_col, housing_identifier_col,
                                person_identifier_col)

  #todo:delete
  df.drop([housing_identifier, person_identifier],axis=1,inplace=True)
  corr = abs(df.corr(method='pearson', numeric_only=False)) #absolute val
  target_corr = corr[target].sort_values(ascending=False)[1:]
  print(target_corr)

  df.to_csv('/content/drive/My Drive/' + target + "_" + limiter + ".csv", index=False) #save to csv

In [ ]:
def request_vars_and_merge(vars, df, base_request, limiter, target, num_vars, curr,
                           target_col, weight_col, housing_identifier_col,
                           person_identifier_col):
  '''
  df: existing df
  base_request: request for weight, target, & identifiers
  limiter: any limiters on the data requested (e.g. state)
  target: target variable
  num_vars: vars in base_request
  target_col: target var data
  weight_col: weights
  housing_identifier_col: housing ids
  person_identifier_col: person ids within a household
  '''

  #get new vars
  new_df, curr = request_vars(vars, base_request, limiter, target, num_vars, curr)
  new_df[target] = target_col
  new_df[weight] = weight_col
  new_df[housing_identifier] = housing_identifier_col
  new_df[person_identifier] = person_identifier_col

  new_cols = new_df.columns

  #merge with existing df
  df = pd.merge(df, new_df, on=[housing_identifier, person_identifier, weight, target])
  df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

  #pca on whole df (todo)

  return df, curr

In [ ]:
def request_vars(vars, base_request, limiter, target, num_vars, curr):
  #request data
  request = base_request

  #create request
  while num_vars < 50 and curr < len(vars):
    request += ','+vars[curr]
    curr += 1
    num_vars += 1
  request += limiter

  #request
  response = requests.get(request)
  json_data = json.dumps(response.json())
  new_df = pd.read_json(json_data)
  new_df = new_df.rename(columns=new_df.iloc[0]).loc[1:].reset_index(drop=True)

  #drop non-numeric (can't check correlation & don't need to)
  new_df.drop([housing_identifier, person_identifier],axis=1,inplace=True)

  #converting N (meaning N/A) to 0, have individually checked each var to confirm this is true
  new_df.replace('N', 0, inplace = True)

  #indp = based on industry codes, naicsp = based on NAICS codes (indp is
  #derived from naicsp and is less detailed to protect individual respondents)
  #indp also has higher correlation with income so choosing to keep indp over naicsp
  if "NAICSP" in new_df.columns:
    new_df.drop("NAICSP", axis=1, inplace=True)

  #recode SOCP
  if "SOCP" in new_df.columns:
    new_df["SOCP"] = recode(new_df, "SOCP")

  #pearson correlation coefficient analysis w target (PINCP)
  threshold = 0.2 #todo: change?
  corr = abs(new_df.corr(method='pearson', numeric_only=False)) #absolute val
  target_corr = corr[target].sort_values(ascending=False)[1:]
  features = target_corr[target_corr>=threshold]
  print("selected features:") #todo:delete
  print(features)

  #pca (todo)

  #fod1p/fod2p have same info but fod2p doesn't have a high enough correlation so already dropped

  #filter new_df
  new_df = new_df[features.index]
  return new_df, curr

In [ ]:
def recode(df, col):
  '''
  returns recoded col
  '''
  unique = dict(enumerate(df[col].unique()))
  unique = dict([(value, key) for key, value in unique.items()])
  return df[col].replace(unique)

In [ ]:
curr_vars = ["PINCP", "PWGTP", "SERIALNO", "HHLDRAGEP", "SSIP", "ELEP", "RAC2P",
             "RAC3P", "RAC1P", "RACNUM", "WATP", "MHP", "RETP", "SSP", "HINCP",
             "RMSP", "INTP", "SEMP", "SMP", "PERNP", "PAP", "GASP", "WKWN",
             "WAGP", "FULP", "SMOCP", "FINCP", "OIP", "TAXAMT", "CONP",
             "INSP", "OCPIP", "GRNTP", "MRGP", "VALP", "BDSP", "NOC", "NP",
             "NRC", "SPORDER", "NPF", "RNTP", "WKHP", "POVPIP", "GRPIP",
             "JWMNP", "AGEP", "ADJHSG", "ADJINC", "MV", "FPARC", "DRIVESP",
             "RACSOR", "NATIVITY", "JWAP", "HICOV", "PRIVCOV", "R60",
             "RELSHIPP", "VACDUR", "MLPIK", "PLM", "VPS", "DEAR", "R18", "MLPJ",
             "GCL", "STOV", "TEL", "ELEFP", "WATFP", "YOEP", "SMX", "OTHSVCEX",
             "MLPCD", "ANC2P", "FHINS4C", "WRK", "POBP", "RACAIAN", "LAPTOP",
             "HHT2", "MLPFG",
            "FOD1P", "FOD2P", "SMARTPHONE", "NAICSP", "INDP", "WAOB", "SOCP", "GASFP", "HIMRKS",
            "FHINS3C", "FHINS5C", "ACCESSINET", "HOTWAT", "NWLA", "CITWP",
            "JWTRNS", "REFR", "PSF", "DECADE", "PUBCOV", "FULFP", "MRGT",
            "VACOTH", "BROADBND", "LANP", "ANC1P", "TEN", "POWPUMA",
            "HISPEED", "PLMPRP", "CPLT", "YRBLT", "DRAT", "NR", "MRGX", "HINS7",
            "MARHYP", "COMPOTHX", "SINK", "MARHT", "SATELLITE", "WIF",
            "HISP", "MAR", "SCHL", "NWLK", "DPHY", "DEYE", "MIGSP",
            "HHLANP", "PARTNER", "RACNH", "WKL", "VEH", "DDRS", "MIGPUMA", "LNGI",
            "HINS2", "QTRBIR", "SFN", "RACBLK", "MLPH", "ESR", "NPP", "DIS",
            "DIALUP", "HHLDRRAC1P", "TABLET", "MLPB", "DOUT", "SCH",
            "RACPI", "POWSP", "ANC", "MIL", "OC", "HUGCL", "RWAT", "HHLDRHISP",
            "HINS3", "RESMODE", "MARHW", "SFR", "ESP", "RACASN", "HINS5", "MLPE",
            "OCCP", "MARHD", "SCHG", "MRGI", "MIG", "HINS1", "MSP", "FER",
            "MULTG", "WORKSTAT", "MARHM", "KIT", "GCR", "HUPARC", "HINS6",
            "GCM", "ACR", "HINS4", "PAOC", "RNTM", "DRATX", "FS", "SVAL",
            "RACWHT", "NWAB", "HUPAOC", "R65", "RC", "BATH", "SEX", "HFL",
            "WKEXREL", "VACS", "HHL", "SRNT", "NWAV", "NWRE", "BLD", "LANX",
            "MLPA", "HHT", "DREM", "COW", "HUPAC", "CIT", "AGS", "ENG", "JWRIP",
            "JWDP", "NOP"]

In [ ]:
#aapi
limiters = {"RACASN":"1", "RACPI":"1", "ST":"0400000US06"} #limting to CA
#limiters = {"RACASN":"1", "RACPI":"1"} #entire U.S.
#limiters = {"ST":"0400000US06"} #all races in CA #todo:check correlation w race
import_census_data(curr_vars, "PINCP", "2021", limiters)

In [ ]:
#predict race
import_census_data(curr_vars, "RAC1P", "2021", "&ucgid=0400000US06")

In [ ]:
#income across race
'''years = ["2005", "2007", "2009", "2011", "2013", "2015", "2017", "2019", "2021"]
#todo:u need to recode NAICSP to INDP (i think hg was helping with that) for this

for y in years:
  import_census_data(curr_vars, "PINCP", y, )''' #need to insert geographies, also probably need a diff vars list bc not all vars every year

#there's going to be inflation for income (can compare income buckets)